[toc]

# Gensim Word2Vec 实战 CallbackAny2Vec 的使用

## 使用 CallbackAny2Vec 打印 loss

In [10]:
from gensim.models.word2vec import Word2Vec
from gensim.models.callbacks import CallbackAny2Vec
raw_sentences = ["What is your name", "My name is edward"]
sentences = [sentence.split() for sentence in raw_sentences]

class PrintLossCallback(CallbackAny2Vec):
    '''Callback to print loss after each epoch.'''

    def __init__(self):
        self.epoch = 0
        self.loss_to_be_subed = 0

    def on_epoch_end(self, model):
        loss = model.get_latest_training_loss()
        loss_now = loss - self.loss_to_be_subed
        self.loss_to_be_subed = loss
        print('Loss after epoch {}: {}'.format(self.epoch, loss_now))
        self.epoch += 1


model = Word2Vec(min_count=1)
model.build_vocab(sentences)
model.train(sentences, total_examples=len(sentences), epochs=10, compute_loss=True, callbacks=[PrintLossCallback()])

Loss after epoch 0: 3.5260963439941406
Loss after epoch 1: 0.0
Loss after epoch 2: 2.8208770751953125
Loss after epoch 3: 2.1156578063964844
Loss after epoch 4: 4.231315612792969
Loss after epoch 5: 0.0
Loss after epoch 6: 2.8208770751953125
Loss after epoch 7: 2.8208770751953125
Loss after epoch 8: 8.462631225585938
Loss after epoch 9: 0.0


(8, 80)

## 使用 CallbackAny2Vec 实现 EarlyStop

In [21]:
from gensim.models.word2vec import Word2Vec
from gensim.models.callbacks import CallbackAny2Vec
import time

raw_sentences = ["What is your name", "My name is edward"]
sentences = [sentence.split() for sentence in raw_sentences]

class OverfitException(Exception):
    pass

class EarlyStopCallBack(CallbackAny2Vec):

    def __init__(self, save_model_name, tolerance=20, delta=0.5):
        self.save_model_name = save_model_name
        self.delta = delta
        self.tolerance = tolerance
        self.loss_to_be_subed = None
        self.best_loss = None
        self.time = None
        self.epoch = 0
        self.best_loss = None
        self.best_epoch = None
        self.counter = 0

    def on_epoch_end(self, model):
        loss_now = model.get_latest_training_loss()
        time_now = time.time()
        if self.loss_to_be_subed is None:
            self.loss_to_be_subed = loss_now
            self.time = time_now
        else:
            loss = loss_now - self.loss_to_be_subed
            print('Loss after epoch {}: {:.3f} time_used: {:.4f}s'.format(self.epoch, loss, time_now - self.time))

            if self.best_loss is None or self.best_loss - loss > self.delta:
                self.best_loss = loss
                self.best_epoch = self.epoch
                model.save(self.save_model_name)
                self.counter = 0
            else:
                self.counter += 1

            if self.counter >= self.tolerance:
                print("Early Stop at epoch {}".format(self.best_epoch))
                raise OverfitException

            self.loss_to_be_subed = loss_now
            self.time = time_now
        self.epoch += 1

model = Word2Vec(min_count=1)
model.build_vocab(sentences)
try:
    model.train(sentences, total_examples=len(sentences), epochs=10, compute_loss=True, callbacks=[EarlyStopCallBack("w2v.model", tolerance=3)])
except OverfitException:
    print("Train finished!")

Loss after epoch 1: 0.000 time_used: 0.0009s
Loss after epoch 2: 2.821 time_used: 0.0018s
Loss after epoch 3: 2.116 time_used: 0.0008s
Loss after epoch 4: 4.231 time_used: 0.0007s
Early Stop at epoch 1
Train finished!


## 使用 CallBackAny2Vec 实现 shuffle 操作

在 item2vec 中，作者提出可以使用 word2vec 的框架，只不过每个 epoch 时进行一次 shuffle 操作。如果是直接使用 gensim 的 word2vec，可以使用 CallBackAny2Vec 来实现 shuffle 操作

In [33]:
from gensim.models.word2vec import Word2Vec
from gensim.models.callbacks import CallbackAny2Vec
import random

raw_sentences = ["What is your name", "My name is edward"]
sentences = [sentence.split() for sentence in raw_sentences]

class ShuffleCallBack(CallbackAny2Vec):
    def __init__(self, sentences):
        self.sentences = sentences
        
    def on_epoch_begin(self, model):
        for i in range(len(self.sentences)):
            random.shuffle(sentences[i]) # 注意shuffle操作应该是inplace的。
            
model = Word2Vec(min_count=1)
model.build_vocab(sentences)
model.train(sentences, total_examples=len(sentences), epochs=10, compute_loss=True, callbacks=[ShuffleCallBack(sentences)])

(4, 80)

# Reference
1. [gensim word2vec print log loss - Stack Overflow](https://stackoverflow.com/questions/54888490/gensim-word2vec-print-log-loss?noredirect=1)
2. [python - How to break the Word2vec training from a callback function? - Stack Overflow](https://stackoverflow.com/questions/58134062/how-to-break-the-word2vec-training-from-a-callback-function)